# Sentiment Classification using pre-trained word embeddings and a LSTM RNN


1. Uses Word2vec word embeddings: 94.15% (2 epochs)
2. Learns own weights during training 92.93% (2 epochs)
3. Uses GLoVe word embeddings: 94.29% (2 epochs)

*Approach:*

Convert all text samples in the dataset into sequences of word indices. A "word index" would simply be an integer ID for the word. We will only consider the top 20,000 most commonly occuring words in the dataset, and we will truncate the sequences to a maximum length of 1000 words.

Prepare an "embedding matrix" which will contain at index i the embedding vector for the word of index i in our word index.

Load this embedding matrix into a Keras Embedding layer, set to be frozen (its weights, the embedding vectors, will not be updated during training).

Build on top of it a LSTM recurrent neural network.

In [ ]:
from __future__ import print_function
import os
import sys
import numpy as np
import pandas as pd
import pickle
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import gensim

BASE_DIR = 'C:/Users/James/Desktop/NLP Summer/'
EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin'
GLOVE_DIR = BASE_DIR + '/glove.42B.300d/'
TRAIN_DATA_FILE = BASE_DIR + '/Yelp/train_Even82k.csv'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

Create text samples and labels and format into tensors that can be fed into a neural network. 
To do this, we will rely on Keras utilities keras.preprocessing.text.Tokenizer and keras.preprocessing.sequence.pad_sequences
as well as an external text processor (text_to_wordlist).

In [ ]:
# second, prepare text samples and their labels
print('Processing text dataset')

# The function "text_to_wordlist" is from
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text

# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

Data Preparation

In [ ]:
#create arrays for text data and labels

texts = [] 
labels = []
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts.append(text_to_wordlist(values[1]))
        labels.append(int(values[0]))
print('Found %s texts in train.csv' % len(texts))

#store and load using pickle
pickle.dump((texts, labels), open("texts_labels_index.pkl", 'wb'))
(texts, labels) = pickle.load(open("texts_labels_index.pkl", 'rb'))


tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

# Approach 1: using Word2Vec weights

In [ ]:
## index word vectors

print('Indexing word vectors')

word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

In [ ]:
## prepare embeddings
########################################
print('Preparing embedding matrix')

nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix_w2v = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix_w2v[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix_w2v, axis=1) == 0))

In [ ]:
embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix_w2v],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)

In [ ]:
num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25
act = 'relu'

In [ ]:
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = lstm_layer(embedded_sequences)
x = Dropout(rate_drop_dense)(x)
x = BatchNormalization()(x)
x = Dense(num_dense, activation=act)(x)
x = Dropout(rate_drop_dense)(x)
x = BatchNormalization()(x)

preds = Dense(1, activation='sigmoid')(x)

In [ ]:
## train the model

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
model.summary()

In [ ]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_val, y_val, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Approach 2: Initializing Embedding layer from scratch and learning its weights during training

We can also test how well we would have performed by not using pre-trained word embeddings, but instead initializing our Embedding layer from scratch and learning its weights during training. We just need to replace our Embedding layer with the following:

In [ ]:
embedding_layer_self = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH)

# Hence, no weights paramenter

In [ ]:
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer_self(sequence_input)
x = lstm_layer(embedded_sequences)
x = Dropout(rate_drop_dense)(x)
x = BatchNormalization()(x)
x = Dense(num_dense, activation=act)(x)
x = Dropout(rate_drop_dense)(x)
x = BatchNormalization()(x)

preds = Dense(1, activation='sigmoid')(x)

In [ ]:
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
model.summary()

In [ ]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_val, y_val, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Approach 3: Use GloVe Weights:

Next, we compute an index mapping words to known embeddings (GloVe), by parsing the data dump of pre-trained embeddings:

In [ ]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.42B.300d.txt'), encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
## prepare embeddings

print('Preparing embedding matrix')

num_words = min(MAX_NB_WORDS, len(word_index)) + 1
embedding_matrix_glove = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_glove[i] = embedding_vector


In [ ]:
# load pre-trained word embeddings into an Embedding layer works
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix_glove],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')

Develop LSTM Model

In [ ]:
num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25
act = 'relu'

In [ ]:
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = lstm_layer(embedded_sequences)
x = Dropout(rate_drop_dense)(x)
x = BatchNormalization()(x)
x = Dense(num_dense, activation=act)(x)
x = Dropout(rate_drop_dense)(x)
x = BatchNormalization()(x)

preds = Dense(1, activation='sigmoid')(x)

In [ ]:
## train the model

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
#model.summary()


In [ ]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_val, y_val, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))